# Evaluation of the novel methods

In [1]:
import numpy as np
from sklearn.model_selection import RepeatedKFold
X = np.array([[1, 2], [3, 4], [1, 2], [3, 4]])
random_state = 0
rkf = RepeatedKFold(n_splits=2, n_repeats=2, random_state=random_state)
for train, test in rkf.split(X):
    print("%s %s" % (train, test))

[0 1] [2 3]
[2 3] [0 1]
[1 3] [0 2]
[0 2] [1 3]


In [23]:
import pandas as pd

X = pd.read_csv("./data/2-EMOT_X.csv")

X, X.columns

(     Unnamed: 0  Mean_Acc1298_Mean_Mem40_Centroid  \
 0             0                          0.034741   
 1             1                          0.081374   
 2             2                          0.110545   
 3             3                          0.042481   
 4             4                          0.074550   
 ..          ...                               ...   
 588         588                          0.027142   
 589         589                          0.094829   
 590         590                          0.035169   
 591         591                          0.054276   
 592         592                          0.073194   
 
      Mean_Acc1298_Mean_Mem40_Rolloff  Mean_Acc1298_Mean_Mem40_Flux  \
 0                           0.089665                      0.091225   
 1                           0.272747                      0.085733   
 2                           0.273567                      0.084410   
 3                           0.199281                      0.09344

In [24]:
y = pd.read_csv("./data/2-EMOT_y.csv")

y, y.columns

(     Unnamed: 0  amazed-suprised  happy-pleased  relaxing-clam  quiet-still  \
 0             0                0              1              1            0   
 1             1                1              0              0            0   
 2             2                0              1              0            0   
 3             3                0              0              1            0   
 4             4                0              0              0            1   
 ..          ...              ...            ...            ...          ...   
 588         588                0              0              1            1   
 589         589                1              0              0            0   
 590         590                0              0              1            1   
 591         591                0              1              1            0   
 592         592                0              1              0            0   
 
      sad-lonely  angry-aggresive  
 0

In [22]:
from sklearn.multioutput import ClassifierChain
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import hamming_loss
from sklearn.model_selection import RepeatedKFold

random_state = 0
rkf = RepeatedKFold(n_splits=5, n_repeats=1, random_state=random_state)
for train_idx, test_idx in rkf.split(X.head(50), y.head(50)):
    X_train = X.iloc[train_idx]
    X_test = X.iloc[test_idx]
    y_train = y.iloc[train_idx]
    y_test = y.iloc[test_idx]

    print(X_train)

    base = LogisticRegression(solver="liblinear", random_state=random_state)
    chain = ClassifierChain(base)

    chain.fit(X_train, y_train)

    y_pred = chain.predict(X_test)
    print(hamming_loss(y_test, y_pred))

    Unnamed: 0  Mean_Acc1298_Mean_Mem40_Centroid  \
0            0                          0.034741   
1            1                          0.081374   
3            3                          0.042481   
5            5                          0.052434   
6            6                          0.064067   
7            7                          0.044949   
8            8                          0.081354   
9            9                          0.039819   
12          12                          0.112665   
13          13                          0.031987   
14          14                          0.056384   
15          15                          0.081393   
16          16                          0.036562   
17          17                          0.073788   
18          18                          0.072026   
19          19                          0.030467   
20          20                          0.090841   
21          21                          0.058040   
23          

ValueError: multiclass-multioutput is not supported